# Loading satellites to program

In [1]:
from skyfield.api import load, wgs84

stations_url = 'http://celestrak.com/NORAD/elements/stations.txt'
satellites = load.tle_file(stations_url)
print('Loaded', len(satellites), 'satellites')

[#################################] 100% stations.txt


Loaded 69 satellites


# making a dictionary with name as their key

In [3]:

by_name = {sat.name: sat for sat in satellites}


In [25]:
# Lookup by name
satellite = by_name['ISS (ZARYA)']
print(satellite)


ISS (ZARYA) catalog #25544 epoch 2021-01-20 05:38:49 UTC


In [49]:
for i in by_name:
    print(i)

ISS (ZARYA)
KESTREL EYE IIM (KE2M)
DELLINGR (RBLE)
TEMPEST-D
RADSAT-G
AEROCUBE 12A
AEROCUBE 12B
LEMUR-2-VU
LEMUR-2-ALEXANDER
LEMUR-2-YUASA
LEMUR-2-TOMHENDERSON
1998-067PN
1998-067PP
STARS-ME
ISS DEB (SEDA-AP)
CATSAT-2
DELPHINI
UNITE
CATSAT-1
ISS DEB
RAAVANA-1
UGUISU
NEPALISAT-1
SPOOQY-1
RED-EYE 1 (PINOT)
IOD-1 GEMS
SWIATOWID
KRAKSAT
VCC A
ENTRYSAT
VCC C
VCC B
RWASAT-1
AQT-D
NARSSCUBE-1
STPSAT-4
HARP
1998-067RA
PHOENIX
1998-067RC
CRYOCUBE
AZTECHSAT-1
RADSAT-U
QARMAN
SORTIE
ICS-EF (ISS DEB)
PROGRESS-MS 14
G-SAT
QUETZAL-1
RED-EYE 2 (MERLOT)
RED-EYE 3 (CABERNET)
DEMI
TECHEDSAT 10
PROGRESS-MS 15
CYGNUS NG-14
SOYUZ-MS 17
CREW DRAGON 1
SPOC
BOBCAT-1
NEUTRON-1
1998-067RU
LEMUR-2-BAXTER-OLIVER
LEMUR-2-DJARA
DESCENT


# Making their number as key

In [19]:
by_number = {sat.model.satnum: sat for sat in satellites}
satellite = by_number[25544]
print(satellite)

ISS (ZARYA) catalog #25544 epoch 2021-01-20 05:38:49 UTC


# Perform queries

In [16]:
n = 25544 #Satllite number
url = 'https://celestrak.com/satcat/tle.php?CATNR={}'.format(n)
filename = 'tle-CATNR-{}.txt'.format(n)
satellites = load.tle_file(url, filename=filename, reload=True)
print(satellites)

[#################################] 100% tle-CATNR-25544.txt


[<EarthSatellite ISS (ZARYA) catalog #25544 epoch 2021-01-20 05:38:49 UTC>]


In [20]:
print(satellite.epoch.utc_jpl())

A.D. 2021-Jan-20 05:38:49.3230 UTC


# Loading form Strings

In [22]:
from skyfield.api import EarthSatellite

ts = load.timescale()
line1 = '1 25544U 98067A   14020.93268519  .00009878  00000-0  18200-3 0  5082'
line2 = '2 25544  51.6498 109.4756 0003572  55.9686 274.8005 15.49815350868473'
satellite = EarthSatellite(line1, line2, 'ISS (ZARYA)', ts)
print(satellite)

ISS (ZARYA) catalog #25544 epoch 2014-01-20 22:23:04 UTC


## Checking epoch

In [40]:
print(satellite.epoch.utc_jpl())

A.D. 2021-Jan-20 05:38:49.3230 UTC


## Rise and set of satellite

In [38]:
bluffton = wgs84.latlon(+27.3314, +88.6138)
t0 = ts.utc(2021, 1, 20)
t1 = ts.utc(2021, 1, 21)
t, events = satellite.find_events(bluffton, t0, t1, altitude_degrees=30.0)
for ti, event in zip(t, events):
    name = ('rise above 30°', 'culminate', 'set below 30°')[event]
    print(ti.utc_strftime('%Y %b %d %H:%M:%S'), name)

2021 Jan 20 11:25:36 rise above 30°
2021 Jan 20 11:27:01 culminate
2021 Jan 20 11:28:27 set below 30°
2021 Jan 20 19:36:58 rise above 30°
2021 Jan 20 19:38:20 culminate
2021 Jan 20 19:39:42 set below 30°


## position of satellite

In [39]:
t = ts.now()

geocentric = satellite.at(t)
print(geocentric.position.km) # x,y,z coordinates from Earths center as reference

[-2708.99494262  4072.17310385  4711.43050537]


In [42]:
subpoint = wgs84.subpoint(geocentric)
print('Latitude:', subpoint.latitude)
print('Longitude:', subpoint.longitude)
print('Elevation (m):', int(subpoint.elevation.m))

Latitude: 44deg 02' 43.9"
Longitude: -149deg 43' 45.0"
Elevation (m): 423240


In [43]:
difference = satellite - bluffton
print(difference)

Sum of 2 vectors:
 Reversed Geodetic WGS84 latitude 27deg 19' 53.0" N longitude 88deg 36' 49.7" E -> 399 EARTH
 EarthSatellite 399 EARTH -> ISS (ZARYA) catalog #25544 epoch 2021-01-20 05:38:49 UTC


In [45]:
topocentric = difference.at(t)
print(topocentric.position.km)

[-8381.27357517  3867.43264488  1811.97149357]


## topocentric position using altitude and azimuth coordinates

In [46]:

alt, az, distance = topocentric.altaz()

if alt.degrees > 0:
    print('The ISS is above the horizon')

print(alt)
print(az)
print(int(distance.km), 'km')

-43deg 40' 24.2"
37deg 46' 27.4"
9406 km


## if satellite is in sunlight

In [68]:
eph = load('de421.bsp')
satellite = by_name['RED-EYE 3 (CABERNET)']

two_hours = ts.utc(2021, 1, 20, 11, range(0, 120, 20))
sunlit = satellite.at(two_hours).is_sunlit(eph)
print(sunlit)

[False  True  True  True False False]


In [54]:
print(satellite.epoch.utc_jpl())

A.D. 2021-Jan-20 04:16:54.4253 UTC


In [66]:
ts.now().utc_strftime(format='%Y-%m-%d %H:%M:%S UTC')


'2021-01-20 11:26:09 UTC'

In [56]:
ts.utc(0,0,0)

<Time tt=1721027.5004882407>

In [71]:
two_hours = ts.utc(2021, 1, 20, 11, range(0, 120, 20))
for ti, sunlit_i in zip(two_hours, sunlit):
    print('{}  {} is in {}'.format(
        ti.utc_strftime('%Y-%m-%d %H:%M'),satellite.name,'sunlight' if sunlit_i else 'shadow'))

2021-01-20 11:00  RED-EYE 3 (CABERNET) is in shadow
2021-01-20 11:20  RED-EYE 3 (CABERNET) is in sunlight
2021-01-20 11:40  RED-EYE 3 (CABERNET) is in sunlight
2021-01-20 12:00  RED-EYE 3 (CABERNET) is in sunlight
2021-01-20 12:20  RED-EYE 3 (CABERNET) is in shadow
2021-01-20 12:40  RED-EYE 3 (CABERNET) is in shadow


# Find whether the Earth blocks a satellite’s view

In [73]:
ph = load('de421.bsp')
earth, venus = eph['earth'], eph['venus']
satellite = by_name['RED-EYE 3 (CABERNET)']

two_hours = ts.utc(2021, 1, 20, 11, range(0, 120, 20))
p = (earth + satellite).at(two_hours).observe(venus).apparent()
sunlit = p.is_behind_earth()
print(sunlit)

[ True False False False  True  True]
